# MD Analysis functions for MD4224

In [1]:
print '\n\tImporting MD Analysis functions for MD4224'
import os
import sys
import gzip
import glob
import pickle
#import imageio
import pandas as pd
import numpy as np
import scipy.io as sio 

sys.path.append('/eos/user/s/sterbini/MD_ANALYSIS/public/')
sys.path.append('/eos/project/l/liu/Toolbox/')

from myToolbox import *
import matplotlib.cm as cm
from matplotlib.lines import Line2D
from math import log10, floor
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
import matplotlib.gridspec as gridspec    


	Importing MD Analysis functions for MD4224
Version 0.03. This is the latest version.
Please help me to improve it reporting bugs to guido.sterbini@cern.ch.
Your platform is Linux-3.10.0-1127.10.1.el7.x86_64-x86_64-with-centos-7.7.1908-Core
Your folder is /eos/home-h/harafiqu/SWAN_projects/PS/From_Scratch
Your IP is 172.17.0.14
2020-08-14 13:34:31


In [2]:
def make_directory(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)  

## Create a dataframe from saved matlab MD files

In [3]:
def fromMatlabToDataFrame(listing, listOfVariableToAdd, verbose=False, matlabFullInfo=False):
    listOfVariableToAdd=list(set(listOfVariableToAdd))
    myDataFrame=pnd.DataFrame({})
    cycleStamp=[]
    cycleStampHuman=[]
    PLS_matlab=[]
    matlabObject=[]
    matlabFilePath=[]
    for j in listOfVariableToAdd:
        exec(j.replace('.','_')+'=[]')
    for i in listing:
        if verbose:
            print(i)
        data=myToolbox.japcMatlabImport(i);
        if matlabFullInfo:
            matlabObject.append(data)
        #to correct
        localCycleStamp=np.nanmax(data.headerCycleStamps);
        deltaLocal_UTC=datetime.datetime.fromtimestamp(localCycleStamp/1e9)-datetime.datetime.utcfromtimestamp(localCycleStamp/1e9)
        utcCycleStamp=localCycleStamp+deltaLocal_UTC.total_seconds()*1e9
        cycleStamp.append(utcCycleStamp)
        aux=myToolbox.unixtime2datetimeVectorize(np.nanmax(data.headerCycleStamps)/1e9)
        cycleStampHuman.append(aux.tolist())
        PLS_matlab.append(data.cycleName)
        matlabFilePath.append(os.path.abspath(i))
        for j in listOfVariableToAdd:
            if hasattr(data,j.split('.')[0]):
                exec(j.replace('.','_') + '.append(data.' + j + ')')
            else:
                exec(j.replace('.','_') + '.append(np.nan)')
    myDataFrame['cycleStamp']=pnd.Series(cycleStamp,cycleStampHuman)
    myDataFrame['matlabPLS']=pnd.Series(PLS_matlab,cycleStampHuman)
    myDataFrame['matlabFilePath']=pnd.Series(matlabFilePath,cycleStampHuman)
    if matlabFullInfo:
        myDataFrame['matlabFullInfo']=pnd.Series(matlabObject,cycleStampHuman)
    for j in listOfVariableToAdd:
        exec('myDataFrame[\'' + j + '\']=pnd.Series(' +j.replace('.','_')+ ',cycleStampHuman)')    #myDataFrame=pnd.DataFrame({j:aux,
    return myDataFrame

## Calculate maximum intensity, losses, and ratio from dataframe MD data

In [4]:
def intensity_df(ndf,start=85,end=1185):
    df=ndf.copy()
    def losses(a):
        try:
            b=(a[start]-a[end])/a[start]*100 
        except:
            b=np.nan
        return b
    def ratio(a):
        try:
            c=a[end]/a[start]
        except:
            c=np.nan
        return c
    def max_intensity(a):
        try:
            d=np.max(a)
        except:
            d=np.nan
        return d*1E10
    df['intensity']=df['PR_BCT_ST.Samples.value.samples'].apply(max_intensity)
    df['losses']=df['PR_BCT_ST.Samples.value.samples'].apply(losses)
    df['ratio']=df['PR_BCT_ST.Samples.value.samples'].apply(ratio)
    return df

## Read data from Timber

In [5]:
def read_data(year=2018,date=4,month=9,hour=8,minutes=20,span=8,user='CPS%MD1',WS='65.H'):
    t1=datetime.datetime(year,month,date,hour,minutes)
    t2=t1+datetime.timedelta(hours=span)
    CALS=['PR.BWS.{}_ROT:PROF_DATA_IN'.format(WS),  'PR.BWS.{}_ROT:PROF_POSITION_IN'.format(WS),  'PR.BWS.{}_ROT:ACQ_DELAY'.format(WS), 'PR.BCT.LT:SAMPLES']
    df=myToolbox.fromTimberToDataFrame(CALS,t1,t2,fundamental=user)
    return  df

## Gaussian fits for 3, 4, and 5 parameters

In [6]:
def gaussian_3_parameters(x, A, mu, sig):
    """gaussian_3_parameter(x, m, A, mu, sig)"""
    return A/np.sqrt(2*np.pi)/sig*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def gaussian_4_parameters(x, m, A, mu, sig):
    """gaussian_4_parameter(x, m, A, mu, sig)"""
    return m*x+A/np.sqrt(2*np.pi)/sig*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def gaussian_5_parameters(x, c, m, A, mu, sig):
    """gaussian_5_parameter(x, c, m, A, mu, sig)"""
    return c+m*x+A/np.sqrt(2*np.pi)/sig*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

## Calculate second moment of a distribution

In [7]:
def second_moment(values, weights):
    weighted_average = np.average(values, weights=weights)
    second_moment = np.sqrt(np.average((values-weighted_average)**2, weights=weights))
    return (second_moment)

def second_moment_2(values, weights, sig, n_sigmas=6.):
    a=np.where((values<n_sigmas*sig) & (values>-n_sigmas*sig))[0]
    values=values[a]
    weights=weights[a]    
    weighted_average = np.average(values, weights=weights)
    second_moment = np.sqrt(np.average((values-weighted_average)**2, weights=weights))
    return (second_moment)

## Apply a svipy.savgol filter to data

In [8]:
def filtered(a):
    try:
        b=(scipy.signal.savgol_filter(a,15,1))
    except:
        b=a
    return b

## Multi step filtering of wirescanner profiles to extract standard deviation sigma and second moment mu

In [9]:
def Multi_filter(ndf, with_plot = False, WS = '65.H', sig_limit_mm=10, smooth = False):
    
    df = ndf.copy()
    
    position_label = 'PR.BWS.'+WS+'_ROT:PROF_POSITION_IN'
    profile_label = 'PR.BWS.'+WS+'_ROT:PROF_DATA_IN'
    
    window = 40    
    
    # Arrays to hold dataframe data. Appended to dataframe at the end of the loop.
    mu_5 = []
    sigma_5 = []
    A_5 = []
    m_5 = []
    c_5 = []
    sig_err_5 = []
    
    mu_3 = []
    sigma_3 = []
    A_3 = []
    sig_err_3 = []
    
    mom_2nd = []
    
    filtered_profiles = []
    filtered_positions = []
    
    failed_rows = []
    
    new_new_X=[]
    
    # Iterate over dataframe
    for j in range(0, len(df), 1):
        print '\nMulti_filter row ', j
        
# 5 parameter Gaussian fit to find mean and sigma
#------------------------------------------------
    
        data = []
        # Filter the data (scipy.signal.savgol_filter)
        if smooth: data = filtered(df[profile_label].iloc()[j])    
        else:data = df[profile_label].iloc()[j]
            
        X = df[position_label].iloc()[j]/1000.
        Y = data
        indx_max = np.argmax(Y)
        mu0 = X[indx_max]
        x_tmp = X
        y_tmp = Y
        offs0 = min(y_tmp)
        ampl = max(y_tmp)-offs0
        x1 = x_tmp[np.searchsorted(y_tmp[:window], offs0+ampl/2)]
        x2 = x_tmp[np.searchsorted(-y_tmp[window:], -offs0+ampl/2)]
        FWHM = x2-x1
        #sigma0 = np.abs(2*FWHM/2.355)
        sigma0 = 5
        ampl *= np.sqrt(2*np.pi)*sigma0
        slope = 0
        
        popt5 = []
        pcov5 = []    

        popt5, pcov5 = curve_fit(gaussian_5_parameters, X, data, p0 =[offs0, slope, ampl, mu0, sigma0])
        result5 = gaussian_5_parameters(X, popt5[0], popt5[1], popt5[2], popt5[3], popt5[4])

        fit_c_5 = popt5[0]
        fit_m_5 = popt5[1]
        fit_A_5 = popt5[2]
        fit_mean_5 = popt5[3]
        fit_sigma_5 = abs(popt5[4])
        
        c_5.append(np.array(popt5[0]))
        m_5.append(np.array(popt5[1]))
        A_5.append(np.array(popt5[2]))
        mu_5.append(np.array(popt5[3]))
        sigma_5.append(abs(np.array(abs(popt5[4]))))
        sig_err_5.append(np.sqrt(np.diag(pcov5[4])[4][4]))
        
        print '5 parameter fit sigma = ', fit_sigma_5 , ' +/- ', np.sqrt(np.diag(pcov5[4]))[4][4]
    
        if fit_sigma_5 > sig_limit_mm:
            
            print '\n\tinitial guess: mu0', mu0
            print '\tinitial guess: offs0', offs0
            print '\tinitial guess: ampl', ampl
            print '\tinitial guess: slope', slope
            print '\tinitial guess: sigma0', sigma0
            
            result0 = gaussian_5_parameters(X, offs0, slope, ampl, mu0, sigma0)
            
            fig, ax1 = plt.subplots(figsize=(8, 4), dpi= 200, facecolor='w', edgecolor='k');

            plt.plot(X, df[profile_label].iloc()[j], label='raw');
            plt.plot(X, result5, label='5 param fit');
            plt.plot(X, result0, label='initial param fit');

            tit = 'WS profile & fit ' + str(j);
            savename = 'MD4224_HB_Vertical_Multifit_' + str(j);

            plt.title(tit);
            plt.ylabel('Profile [-]');
            plt.xlabel('Position [mm]');
            plt.legend();            
            plt.grid();
            fig.savefig(savename);    
            plt.close();
            
            # Set empty array values for completeness - cannot set dataframe with missing values
            filtered_profiles.append(np.nan)
            filtered_positions.append(np.nan)
            
            failed_rows.append(j)

            mom_2nd.append(np.nan)

            c_5.append(np.nan)
            m_5.append(np.nan)
            A_5.append(np.nan)
            mu_5.append(np.nan)
            sigma_5.append(np.nan)
            sig_err_5.append(np.nan)

            A_3.append(np.nan)
            mu_3.append(np.nan)
            sigma_3.append(np.nan)
            sig_err_3.append(np.nan)        
  
            
        # Simple filter for negative sigma (fit didn't work)
        elif fit_sigma_5 < sig_limit_mm:
            
# Consider only data between +/-(6 sigma + mean)
#------------------------------------------------
            new_X = []
            new_Y = []        

            # Iterate over data
            for i in xrange(len(Y)): # Append data if inside cut window
                if (X[i] < (6*fit_sigma_5 + fit_mean_5)) and (X[i] > (-6*fit_sigma_5 + fit_mean_5)):

# Remove Slope and Offset
#------------------------------------------------               
                    new_X.append(X[i])
                    new_Y.append(Y[i] - (fit_m_5*X[i] + fit_c_5))

# 3 parameter Gaussian fit to centre
#------------------------------------------------

            popt3 = []
            pcov3 = []   
            popt3, pcov3 = curve_fit(gaussian_3_parameters, new_X, new_Y, p0 =[fit_A_5, fit_mean_5, fit_sigma_5])
            result3 = gaussian_3_parameters(new_X, popt3[0], popt3[1], popt3[2])

            fit_A_3 = popt3[0]
            fit_mean_3 = popt3[1]
            fit_sigma_3 = popt3[2]

            A_3.append(np.array(popt3[0]))
            mu_3.append(np.array(popt3[1]))
            sigma_3.append(np.array(abs(popt3[2])))
            sig_err_3.append(np.sqrt(np.diag(pcov3[2]))[2][2])     

            print '3 parameter fit sigma = ', popt3[2] , ' +/- ', np.sqrt(np.diag(pcov3[2]))[2][2]

# Threshold ?
#------------------------------------------------

# 2nd moment
#------------------------------------------------
            # Shift centre
            new_new_X = new_X - fit_mean_3

            second_mom = second_moment(new_new_X, new_Y)

            mom_2nd.append(second_mom)

            print 'Second moment = ', second_mom        

            # Append to dataframe later
            filtered_positions.append(new_new_X)
            filtered_profiles.append(new_Y)       

# Plot
#------------------------------------------------      
            if with_plot:
                fig, ax1 = plt.subplots(figsize=(8, 4), dpi= 200, facecolor='w', edgecolor='k');

                plt.plot(X, df[profile_label].iloc()[j], label='raw');
                plt.plot(X, result5, label='5 param fit');
                plt.plot(new_new_X, result3, label='3 param fit' );
                plt.plot(new_new_X, new_Y, label='filtered');

                tit = 'WS profile & fit ' + str(j);
                savename = 'MD4224_HB_Vertical_Multifit_' + str(j);

                plt.title(tit);
                plt.ylabel('Profile [-]');
                plt.xlabel('Position [mm]');
                plt.legend();            
                plt.grid();
                fig.savefig(savename);
                plt.close();

    # Set values in dataframe   
    
#     df['5sig '+WS[-1]] = sigma_5
#     df['5sig err'+WS[-1]] = sig_err_5
#     df['5A '+WS[-1]] = A_5
#     df['5mu '+WS[-1]] = mu_5
#     df['5m '+WS[-1]] = m_5
#     df['5c '+WS[-1]] = c_5
    
#     df['3sig '+WS[-1]] = sigma_3
#     df['3sig err'+WS[-1]] = sig_err_3
#     df['3A '+WS[-1]] = A_3
#     df['3mu '+WS[-1]] = mu_3
    
    df['2ndMoment '+WS[-1]] = mom_2nd   
    
    df['sig '+WS[-1]] = sigma_3
    df['sig err'+WS[-1]] = sig_err_3
    df['A '+WS[-1]] = A_3
    df['mu '+WS[-1]] = mu_3
    
    df['position '+WS[-1]] = filtered_positions
    df['profile '+WS[-1]] = filtered_profiles
    
    print 'failed rows = ', failed_rows
    
    return df

## Multi step filtering of simulation profiles to extract standard deviation sigma and second moment mu

In [10]:
def Multi_filter_sim(ndf, with_plot = False, WS = '65.H', sig_limit_mm=10, smooth = False):
    
    df = ndf.copy()
    
    position_label = 'PR.BWS.'+WS+'_ROT:PROF_POSITION_IN'
    profile_label = 'PR.BWS.'+WS+'_ROT:PROF_DATA_IN'
    
    window = 40    
    
    # Arrays to hold dataframe data. Appended to dataframe at the end of the loop.
    mu_5 = []
    sigma_5 = []
    A_5 = []
    m_5 = []
    c_5 = []
    sig_err_5 = []
    
    mu_3 = []
    sigma_3 = []
    A_3 = []
    sig_err_3 = []
    
    mom_2nd = []
    
    filtered_profiles = []
    filtered_positions = []
    
    failed_rows = []
    
    new_new_X=[]
    
    # Iterate over dataframe
    for j in range(0, len(df), 1):
        print '\nMulti_filter row ', j
        
# 5 parameter Gaussian fit to find mean and sigma
#------------------------------------------------

        data = []
        # Filter the data (scipy.signal.savgol_filter)
        if smooth: data = filtered(df[profile_label].iloc()[j])    
        else:data = df[profile_label].iloc()[j]
            
        #X = df[position_label].iloc()[j]/1000.
        X = df[position_label].iloc()[j]*1000
        Y = data
        indx_max = np.argmax(Y)
        mu0 = X[indx_max]
        x_tmp = X
        y_tmp = Y
        offs0 = min(y_tmp)
        ampl = max(y_tmp)-offs0
        x1 = x_tmp[np.searchsorted(y_tmp[:window], offs0+ampl/2)]
        x2 = x_tmp[np.searchsorted(-y_tmp[window:], -offs0+ampl/2)]
        FWHM = x2-x1
        sigma0 = np.abs(2*FWHM/2.355)
        #sigma0 = 5
        ampl *= np.sqrt(2*np.pi)*sigma0
        slope = 0
        
        ampl = peakfinder(np.array(bins_[:-1]), np.array(data))
                        
        print '\n\tinitial guess: mu0', mu0
        print '\tinitial guess: offs0', offs0
        print '\tinitial guess: ampl', ampl
        print '\tinitial guess: slope', slope
        print '\tinitial guess: sigma0', sigma0
        
        popt3 = []
        pcov3 = []   
        popt3, pcov3 = curve_fit(gaussian_3_parameters, X, Y, p0 =[ampl, mu0, sigma0])
        result3 = gaussian_3_parameters(X, popt3[0], popt3[1], popt3[2])

        fit_A_3 = popt3[0]
        fit_mean_3 = popt3[1]
        fit_sigma_3 = popt3[2]

        A_3.append(np.array(popt3[0]))
        mu_3.append(np.array(popt3[1]))
        sigma_3.append(np.array(abs(popt3[2])))
        sig_err_3.append(np.sqrt(np.diag(pcov3[2]))[2][2])     

        print '3 parameter fit sigma = ', popt3[2] , ' +/- ', np.sqrt(np.diag(pcov3[2]))[2][2]
                
# 2nd moment
#------------------------------------------------
        # Shift centre
        new_new_X = X - fit_mean_3
        print 'Removing centre shift of ', fit_mean_3
        
        filtered_positions.append(new_new_X)
        filtered_profiles.append(Y)    

        second_mom = second_moment(new_new_X, Y)

        mom_2nd.append(second_mom)

        print 'Second moment = ', second_mom     
        
        # Plot
#------------------------------------------------      
        if with_plot:
            fig, ax1 = plt.subplots(figsize=(8, 4), dpi= 200, facecolor='w', edgecolor='k');

            plt.plot(X, df[profile_label].iloc()[j], label='raw');
            plt.plot(new_new_X, result3, label='3 param fit' );
            plt.plot(new_new_X, Y, label='filtered');

            tit = 'WS profile & fit ' + str(j);
            savename = 'MD4224_HB_Vertical_Multifit_' + str(j);

            plt.title(tit);
            plt.ylabel('Profile [-]');
            plt.xlabel('Position [mm]');
            plt.legend();            
            plt.grid();
            fig.savefig(savename);
            plt.close();
             
    df['2ndMoment '+WS[-1]] = mom_2nd   
    
    df['sig '+WS[-1]] = sigma_3
    df['sig err'+WS[-1]] = sig_err_3
    df['A '+WS[-1]] = A_3
    df['mu '+WS[-1]] = mu_3
    
    df['position '+WS[-1]] = filtered_positions
    df['profile '+WS[-1]] = filtered_profiles
    
    print 'failed rows = ', failed_rows
    
    return df

## Core filter removes tails and fits gaussian to wirescanner profiles

In [11]:

def Core_filter(ndf, with_plot = True, WS = '65.H', sig_limit_mm=10, smooth = False):
    
    df = ndf.copy()
    
    position_label = 'position H'
    profile_label = 'profile H'
    window = 40
    
    
    mu_3 = []
    sigma_3 = []
    A_3 = []
    sig_err_3 = []
    
    mom_2nd = []
    
    # Iterate over dataframe
    for j in range(0, len(df), 1):
        print '\nCore_filter row ', j    
    
        y_data = df[profile_label].iloc()[j]
        x_data = df[position_label].iloc()[j]
        indx_max = np.argmax(y_data)  
        peak = y_data[indx_max]
        
#         print '\nPeak magnitude ', peak, 'found at index ', indx_max
        
        quart_len_x_dat = int(len(x_data)/4)
        
#         print '\nUsing indices', int(indx_max-quart_len_x_dat), ' to ', int(indx_max+quart_len_x_dat)
        
        # Use only half of the data, around the peak        
#         half_peak_indices = find_half_peak_indices(x_data, y_data)
        # equivalent
        half_peak_indices = find_factor_of_peak_indices(x_data, y_data, 0.5, verbose=True)
        
        X = x_data[half_peak_indices[0] : half_peak_indices[1]]
#         X = X/1000
        Y = y_data[half_peak_indices[0] : half_peak_indices[1]]
        
        indx_max = np.argmax(Y)        
        mu0 = X[indx_max]
        x_tmp = X
        y_tmp = Y
        offs0 = min(y_tmp)
        ampl = max(y_tmp)-offs0
#         x1 = x_tmp[np.searchsorted(y_tmp[:window], offs0+ampl/2)]
#         x2 = x_tmp[np.searchsorted(-y_tmp[window:], -offs0+ampl/2)]
#         FWHM = x2-x1
        #sigma0 = np.abs(2*FWHM/2.355)
        sigma0 = 1
        ampl *= np.sqrt(2*np.pi)*sigma0
        slope = 0
    
        popt3 = []
        pcov3 = []   
        
        try:
            popt3, pcov3 = curve_fit(gaussian_3_parameters, X, Y, p0 =[ampl, mu0, sigma0])

            fit_A_3 = popt3[0]
            fit_mean_3 = popt3[1]
            fit_sigma_3 = popt3[2]

            A_3.append(np.array(popt3[0]))
            mu_3.append(np.array(popt3[1]))
            sigma_3.append(np.array(abs(popt3[2])))
            sig_err_3.append(np.sqrt(np.diag(pcov3[2]))[2][2])   
            
            print '3 parameter fit sigma = ', popt3[2] , ' +/- ', np.sqrt(np.diag(pcov3[2]))[2][2]
# Plot
#------------------------------------------------      
            if with_plot:
                fig, ax1 = plt.subplots(figsize=(8, 4), dpi= 200, facecolor='w', edgecolor='k');

                x_dat = np.linspace(-20, 20, 1000)
                result3 = gaussian_3_parameters(x_dat, popt3[0], popt3[1], popt3[2])

                plt.plot(df[position_label].iloc()[j], df[profile_label].iloc()[j], label='filtered data');
                plt.plot(X, Y, label='raw core data');
                plt.plot(x_dat, result3, label='3 param core fit' );

                tit = 'WS profile & fit ' + str(j);
                savename = 'MD4224_HB_Vertical_Corefit_' + str(j);

                plt.title(tit);
                plt.ylabel('Profile [-]');
                plt.xlabel('Position [mm]');
                plt.legend();            
                plt.grid();
                fig.savefig(savename);
                plt.close();            
            
        except:            
            
            print '3 parameter fit not found'
                        
            result3 = 'NaN'

            fit_A_3 = 'NaN'
            fit_mean_3 = 'NaN'
            fit_sigma_3 = 'NaN'

            A_3.append('NaN')
            mu_3.append('NaN')
            sigma_3.append('NaN')
            sig_err_3.append('NaN')   
        
        second_mom = second_moment(X, Y)

        mom_2nd.append(second_mom)
    
    df['Core 2ndMoment '+WS[-1]] = mom_2nd   
    
    df['Core sig '+WS[-1]] = sigma_3
    df['Core sig err'+WS[-1]] = sig_err_3
    df['Core A '+WS[-1]] = A_3
    df['Core mu '+WS[-1]] = mu_3
    
    return df

In [12]:
def span_range(df,myFiles=None):
    if myFiles:
        span=[i for i in xrange(len(df)) if datetime.datetime.strptime(myFiles[0][-27:-4], '%Y.%m.%d.%H.%M.%S.%f')<df.index[i]<datetime.datetime.strptime(myFiles[-1][-27:-4], '%Y.%m.%d.%H.%M.%S.%f')]
    else:
        span=xrange(len(df))
    return span

## Optics functions at wirescanner locations
TODO: redo simulation (SC) optics

In [13]:
def PS_WSoptics(tunex, tuney, WS='65.V', verbose=False):
    result = 0
    
    #Vertical
    if WS[-1]=='V':
        Qy = [6.1, 6.11, 6.12, 6.13, 6.14, 6.15, 6.16, 6.17, 6.18, 6.19, 6.2, 6.21, 6.22, 6.23, 6.24]
        beta_x_ptc = [12.3917987, 12.40432702, 12.41682545, 12.42928263, 12.44169062, 12.45404385, 12.46633837, 12.4785714, 12.490741, 12.50284583, 12.51488502, 12.52685805, 12.53876466, 12.55060479, 12.56237852]
        beta_y_ptc = [23.63758073, 23.28587814, 23.0095329, 22.78949387, 22.6124809, 22.4689646, 22.35193576, 22.25612762, 22.17750836, 22.11294062, 22.05994741, 22.01654769, 21.98113853, 21.95240934, 21.9292783]
        D_x_ptc = [2.656001885, 2.651032449, 2.646066748, 2.641109387, 2.636163621, 2.631231787, 2.626315585, 2.621416255, 2.61653471, 2.611671616, 2.606827458, 2.602002581, 2.597197226, 2.59241155, 2.587645646]
        D_y_ptc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        
        # HR Sims 28.10.19
        Dx_V =  [2.7238641993955732, 2.7120860629004255, 2.6994508041931335, 2.6863440678031574, 2.672386228651023, 2.6580229615003876, 2.64314556738443, 2.6280882092976365, 2.6129041363713723, 2.59744005103213, 2.5818492232775934, 2.5662886921607866, 2.550424494135266, 2.5343524154736436, 2.51866641467657]
        Dy_V =  [4.367816748122375e-05, 0.00035587086701217597, 0.0003660175570428406, 8.26174877194663e-06, -0.000191423682138625, 5.658288855341201e-06, 0.00014281744599578446, -2.878620105586677e-05, -9.647589487896772e-05, -9.090651784262011e-07, -4.668886732849038e-06, 1.4755109576529703e-05, 1.6328867913236533e-05, 2.007351627844793e-05, -1.1071869433949277e-05]
        Beta_x_V =  [11.445953852723896, 11.510142057939072, 11.575929347922242, 11.643662666888034, 11.714421282518094, 11.78791904681883, 11.862932437305316, 11.936289107196501, 12.00850847462817, 12.081015850533856, 12.15156515013633, 12.221473527902477, 12.289860697803704, 12.355190394931768, 12.418258827986351]
        Beta_y_V =  [28.911819899988416, 27.996762372437498, 27.171729802194253, 26.440881544797726, 25.790346611279418, 25.1990325491867, 24.686162791589517, 24.248284914152663, 23.890035080396505, 23.597342279733034, 23.360243624322617, 23.168868800433106, 23.007663535440482, 22.85906902131989, 22.73172392767982]
        Alpha_x_V =  [0.021341472442704016, 0.02001299826083461, 0.018660752366609214, 0.017267348024435355, 0.015807353862269672, 0.014280173350249396, 0.012714813147292111, 0.011191659680433794, 0.00969925057116425, 0.008207509530218775, 0.006769923106013124, 0.00532764358669049, 0.0039347261542670275, 0.002608967925014358, 0.001333583103170329]
        Alpha_y_V =  [-0.07838658779883619, -0.06186142706417254, -0.046655182336960525, -0.03289429996349511, -0.02035536261873128, -0.008852168427724847, 0.001332783765145378, 0.0101563197348924, 0.01754743094728221, 0.02371638294074064, 0.028870423806189226, 0.033184032486125795, 0.0368729594656826, 0.04018756918984884, 0.043056837363835844]

        beta_x_int = interp1d(Qy, beta_x_ptc)
        beta_y_int = interp1d(Qy, beta_y_ptc, kind='cubic')

        D_y_int = interp1d(Qy, D_y_ptc)
        
        if tuney < 1:
            tuney = 6 + tuney
    
        if tuney > 6.24:
            result = {'Beta_x':beta_x_int(6.24), 'Beta_y':beta_y_int(6.24),'D_y': D_y_int(6.24)}
        elif tuney < 6.10:
            result = {'Beta_x':beta_x_int(6.10), 'Beta_y':beta_y_int(6.10),'D_y': D_y_int(6.10)}
        else:
            result = {'Beta_x':beta_x_int(tuney), 'Beta_y':beta_y_int(tuney),'D_y': D_y_int(tuney)}


    #Horizontal
    else:
        # Data from simulations (PyORBIT bunch twiss analysis of tomo distn from nominal (6.21, 6.24) measured tomo)
        Qx = [6.07, 6.08, 6.09, 6.1, 6.11, 6.12, 6.13, 6.14, 6.15, 6.16, 6.17, 6.18, 6.19, 6.2, 6.21]
        
        # HR Flat files with optimised lattice 28.10.19
        beta_x_PTC = [19.91204908, 20.41311562, 20.80478841, 21.11699894, 21.36981083, 21.57714653, 21.74895351, 21.89251823, 22.01329345, 22.11543602, 22.20216651, 22.27601575, 22.33899757, 22.39273226, 22.43853628]
        beta_y_PTC = [11.79735718, 11.79122428, 11.78512065, 11.77905419, 11.77302966, 11.76704997, 11.76111691, 11.75523152, 11.74939435, 11.74360562, 11.73786533, 11.73217331, 11.72652929, 11.7209329, 11.71538372]
        D_x_PTC = [3.966338945, 3.890361791, 3.825040281, 3.768310512, 3.718497363, 3.674282576, 3.634635336, 3.598747003, 3.565978669, 3.53582111, 3.507864727, 3.481777062, 3.457285906, 3.434166539, 3.412232006]

        # HR Sims 28.10.19
        Q_x_H =  [6.07, 6.08, 6.09, 6.1, 6.11, 6.12, 6.13, 6.14, 6.15, 6.16, 6.17, 6.18, 6.19, 6.2, 6.21]
        Dx_H =  [5.094041737639815, 4.745555457055128, 4.465376676611421, 4.243625666843309, 4.069281878600707, 3.9316158604113913, 3.820341066721708, 3.726204087526766, 3.6461483127488576, 3.5756765668001607, 3.5129668013633237, 3.4563946388233266, 3.404480149204493, 3.3564030388195856, 3.3116558798797184]
        Dy_H =  [1.4750033030587474e-05, 6.467605854338345e-07, -1.0514917113366752e-05, 5.815755472624499e-06, 2.1595046605289424e-05, 5.908728772597862e-06, -9.255017467287862e-06, 1.0035542292204026e-05, 1.051816255640615e-05, -1.0728393759677672e-06, 3.9371025222305687e-07, -1.3797255912659215e-05, -1.766537470942474e-05, 7.172375722134988e-06, -1.61589132330799e-05]
        Beta_x_H =  [16.042909156812588, 16.62273975494126, 17.13286856078035, 17.651410896775587, 18.244770290045892, 18.860894438903497, 19.472153382370056, 20.032653880308352, 20.511990757311683, 20.915624794555598, 21.252628685987624, 21.536855772793007, 21.777555988752145, 21.984198721771286, 22.16224322605364]
        Beta_y_H =  [12.852875430115793, 12.806481183737281, 12.764126372010868, 12.71817556261556, 12.66659533313386, 12.614204327190516, 12.562485537918654, 12.511011335754496, 12.46091614678202, 12.415061158200734, 12.37174124057102, 12.332343891998581, 12.29524826325735, 12.259437535414815, 12.22398863524938]
        Alpha_x_H =  [-0.1799963812314074, -0.16250508378742642, -0.14726737076320215, -0.13013227004799938, -0.10913504964040957, -0.08733278265339776, -0.066157270923998, -0.04741824001308929, -0.03178445318085828, -0.018811089794352556, -0.00804469380421407, 0.001044815888149088, 0.008787677241830502, 0.015467148688671589, 0.021290480152427866]
        Alpha_y_H =  [0.11757328235199145, 0.11505359543830002, 0.11269850056765092, 0.1102547485782107, 0.10764803484390695, 0.10507045912893033, 0.10256356772387268, 0.10009810822812605, 0.09773741074883112, 0.09554658412728842, 0.09346903818791467, 0.09150872053178623, 0.08962272899550859, 0.08776747571710704, 0.08592254554093123]

        beta_x_int = interp1d(Qx, beta_x_PTC, kind='cubic')
        beta_y_int = interp1d(Qx, beta_y_PTC)        
        D_x_int = interp1d(Qx, D_x_PTC)
    
        if tunex < 1:
            tunex = 6 + tunex

        if tunex > 6.21:
            result = {'Beta_x':beta_x_int(6.21), 'Beta_y':beta_y_int(6.21),'D_x': D_x_int(6.21)}
        elif tunex < 6.07:
            result = {'Beta_x':beta_x_int(6.07), 'Beta_y':beta_y_int(6.07),'D_x': D_x_int(6.07)}
        else:            
            #print 'Horizontal PS_WSoptics: tune ', tunex, ', ', tuney
            #print 'D_x = ', D_x_int(tunex)
            result = {'Beta_x':beta_x_int(tunex), 'Beta_y':beta_y_int(tunex),'D_x': D_x_int(tunex)}
            #print result
    return result

## Simple 5 parameter gaussian fit of wirescanner data from Foteini

In [14]:
def profiles(ndf,WS='65.H',myFiles=None, lim=40):
    df=ndf.copy()
    # play with these limits
    #x=np.linspace(-40,40,1000)
    x=np.linspace(-lim,lim,1000)
    a=df.columns
    if 'profile '+WS[-1] not in a:
        df['profile '+WS[-1]]=df['PR.BWS.{}_ROT:PROF_DATA_IN'.format(WS)].apply(filtered)
        df['position '+WS[-1]]=df['PR.BWS.{}_ROT:PROF_POSITION_IN'.format(WS)].values/1000.
    if 'sig '+WS[-1] not in a:
        df['sig '+WS[-1]]=np.nan
        df['mu '+WS[-1]]=np.nan
        df['c '+WS[-1]]=np.nan
        df['A '+WS[-1]]=np.nan

    span=span_range(df,myFiles=myFiles)
    if np.isnan(df['sig '+WS[-1]].iloc[span]).all():
        for i in span:
            df['A '+WS[-1]].iloc[i], df['c '+WS[-1]].iloc[i], df['mu '+WS[-1]].iloc[i], df['sig '+WS[-1]].iloc[i] = makeGaussianFit_5_parameters(df['position '+WS[-1]].iloc[i], df['profile '+WS[-1]].iloc[i], for_df=True, window=lim)
            try:
                yz=scipy.interpolate.interp1d(df['position '+WS[-1]].iloc[i] - df['mu '+WS[-1]].iloc[i], df['profile '+WS[-1]].iloc[i] - df['c '+WS[-1]].iloc[i])(x)
                if np.isnan(yz[0]):
                    yz=np.nan
            except:
                yz = np.nan
            df['profile '+WS[-1]].iloc[i]=yz
            df['position '+WS[-1]].iloc[i]=x
    return df

## Add optics parameters to dataframe

In [15]:
def twiss(ndf,WS='65.H',myFiles=None, verbose=False):
    df=ndf.copy()
    a=df.columns
    if 'betx' not in a:
        df['betx']=np.nan
    if 'bety' not in a:
         df['bety']=np.nan
    if 'dx' not in a:
         df['dx']=np.nan
    span=span_range(df,myFiles=myFiles)
    for i in span:
        df.loc[df.index[i],'betx']=PS_WSoptics( df['Qx'].iloc[i], df['Qy'].iloc[i], WS=WS, verbose=verbose )['Beta_x']
        df.loc[df.index[i],'bety']=PS_WSoptics( df['Qx'].iloc[i], df['Qy'].iloc[i], WS=WS, verbose=verbose )['Beta_y']
            
            
        if WS[-1]=='V': df.loc[df.index[i],'dy']=PS_WSoptics( df['Qx'].iloc[i], df['Qy'].iloc[i], WS=WS, verbose=verbose )['D_y']
        elif WS[-1]=='H':
            #print 'result = ', (PS_WSoptics( df['Qx'].iloc[i], df['Qy'].iloc[i], WS=WS, verbose=verbose ))
            #print 'D_x = ', (PS_WSoptics( df['Qx'].iloc[i], df['Qy'].iloc[i], WS=WS, verbose=verbose )['D_x'])
            df.loc[df.index[i],'dx']=PS_WSoptics( df['Qx'].iloc[i], df['Qy'].iloc[i], WS=WS, verbose=verbose )['D_x']

    return df

## Numerous ways of calculating the emittance from dataframe variables

In [16]:
def emittance_df(ndf, WS='65.H', myFiles=None, beta=0.91444281513833, gamma=2.4708737618826, dp=8.7e-4, verbose=False):
    df=ndf.copy()
    a=df.columns
    span=span_range(df,myFiles=myFiles)
    
    if WS[-1]=='V':
        
        if 'emittance V' not in a:
            df['emittance V']=np.nan
        if 'sig V' not in a:
            df=profiles(df,WS=WS)  
        if 'bety' not in a:
            df=twiss(df, WS=WS, verbose=verbose)
        for i in span:
            df.loc[df.index[i],'emittance V']=beta*gamma*(df['sig V'].iloc[i]**2)/df['bety'].iloc[i]
            print 'Emittance ', i, ' = ', beta*gamma*(df['sig V'].iloc[i]**2)/df['bety'].iloc[i]
    else:    
        if 'emittance H' not in a:
            df['emittance H']=np.nan
        if 'sig H' not in a:
            df=profiles(df,WS=WS)
        if 'betx' not in a:
            df=twiss(df,WS=WS)
        for i in span:
            df.loc[df.index[i],'emittance H noDisp']=beta*gamma*(df['sig H'].iloc[i]**2)/df['betx'].iloc[i]
            df.loc[df.index[i],'emittance H']=beta*gamma*(df['sig H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
            print 'Emittance ', i, ' = ', beta*gamma*(df['sig H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
    return df

def emittance_df_2(ndf, WS='65.H', myFiles=None, beta=0.91444281513833, gamma=2.4708737618826, dp=8.7e-4, verbose=False):
    df=ndf.copy()
    a=df.columns
    span=span_range(df,myFiles=myFiles)
    
    if WS[-1]=='V':
        if 'bety' not in a:
            df=twiss(df, WS=WS, verbose=verbose)
        for i in span:
            df.loc[df.index[i],'emittance V 2ndMoment']=beta*gamma*(df['2ndMoment V'].iloc[i]**2)/df['bety'].iloc[i]
            print 'Emittance from 2nd moment', i, ' = ', beta*gamma*(df['2ndMoment V'].iloc[i]**2)/df['bety'].iloc[i]
    else:
        if 'betx' not in a:
            df=twiss(df, WS=WS, verbose=verbose)
        for i in span:
            df.loc[df.index[i],'emittance H 2ndMoment noDisp']=beta*gamma*(df['2ndMoment H'].iloc[i]**2)/df['betx'].iloc[i]
            df.loc[df.index[i],'emittance H 2ndMoment']=beta*gamma*(df['2ndMoment H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
            print 'Emittance from 2nd moment', i, ' = ', beta*gamma*(df['2ndMoment H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
    return df

def core_emittance_df(ndf, WS='65.H', myFiles=None, beta=0.91444281513833, gamma=2.4708737618826, dp=8.7e-4, verbose=False):
    df=ndf.copy()
    a=df.columns    
    
    if WS[-1]=='V':
        if 'Core emittance V' not in a:
            df['Core emittance V']=np.nan
        span=span_range(df,myFiles=myFiles)
        if 'bety' not in a:
            df=twiss(df, WS=WS, verbose=verbose)
        for i in span:
            df.loc[df.index[i],'Core emittance V']=beta*gamma*(df['Core sig V'].iloc[i]**2)/df['bety'].iloc[i]
            print 'Emittance ', i, ' = ', beta*gamma*(df['Core sig V'].iloc[i]**2)/df['bety'].iloc[i]

    else:
        if 'Core emittance H' not in a:
            df['Core emittance H']=np.nan
        span=span_range(df,myFiles=myFiles)
        if 'betx' not in a:
            df=twiss(df, WS=WS, verbose=verbose)
        for i in span:
            df.loc[df.index[i],'Core emittance H noDisp']=beta*gamma*(df['Core sig H'].iloc[i]**2)/df['betx'].iloc[i]
            df.loc[df.index[i],'Core emittance H']=beta*gamma*(df['Core sig H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
            print 'Emittance ', i, ' = ', beta*gamma*(df['Core sig H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
    return df

# Override original due to a bug?
def core_emittance_df_2(ndf, WS='65.H', myFiles=None, beta=0.91444281513833, gamma=2.4708737618826, dp=8.7e-4, verbose=False):
    df=ndf.copy()
    a=df.columns    
    
    if WS[-1]=='V':
        span=span_range(df,myFiles=myFiles)
        if 'bety' not in a:
            df=twiss(df, WS=WS, verbose=verbose)
        for i in span:
            df.loc[df.index[i],'Core emittance V 2ndMoment']=beta*gamma*(df['Core 2ndMoment V'].iloc[i]**2)/df['bety'].iloc[i]
            print 'Emittance ', i, ' = ', beta*gamma*(df['sig V'].iloc[i]**2)/df['bety'].iloc[i]

        
    else:        
        span=span_range(df,myFiles=myFiles)
        if 'betx' not in a:
            df=twiss(df, WS=WS, verbose=verbose)
        for i in span:
            df.loc[df.index[i],'Core emittance H 2ndMoment noDisp']=beta*gamma*(df['Core 2ndMoment H'].iloc[i]**2)/df['betx'].iloc[i]
            df.loc[df.index[i],'Core emittance H 2ndMoment']=beta*gamma*(df['Core 2ndMoment H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
            print 'Emittance ', i, ' = ', beta*gamma*(df['Core 2ndMoment H'].iloc[i]**2-(df['dx'].iloc[i]*1000)**2*dp**2)/df['betx'].iloc[i]
    return df

## function to round a number to n significant figures

In [17]:
def round_sig(x, sig=4):
    return round(x, sig-int(floor(log10(abs(x))))-1)

## load particles from a pyorbit output matfile

In [18]:
def particles_from_matfile(filename):
    d = dict()
    sio.loadmat(filename, mdict=d)
    return d

## function to find peak of wirescanner/simulation profiles

In [19]:
def peakfinder_old (x_dat, y_dat, tolerance = 1, verbose=False):
    
    # Find maximum
    max_y = max(y_dat)
    if verbose: print '\n\tMAX(y_dat) = ', max_y   
    
    # Assume peak as centre
    index = np.where(y_dat == max_y)[0][0]

    # Check if centre is near 0.0
    if abs(x_dat[index]) < tolerance:
        if verbose: print '\tx_dat[index] = ', x_dat[index]
    
    # Manually find 0 index
    else:
        if verbose: print '\tindex from y_dat peak not within tolerance of',tolerance,'mm. \n\ttry manual search'
        
        index = -1
        final_index = -1
        for i in xrange(len(x_dat)):
            index = index + 1
            if x_dat[i] < 0.0:
                final_index = index
                break
                
        index = final_index
        if verbose: print '\tindex = ', index       
       
    if verbose: print '\tindex of x_dat = ', x_dat[index],' is ', index
        
    # Take y=0 +/- 3 points and find mean 
    mean_dat = [y_dat[index-3],  y_dat[index-2],  y_dat[index-1], y_dat[index],  y_dat[index+1],  y_dat[index+2],  y_dat[index+3]]
    mean_7 = np.mean(mean_dat)
    if verbose: print '\tMean of peak point +/- 3 points = ', mean_7   
    
    if abs( abs(mean_7 - max_y) / mean_7 ) > 0.2:
        print '\tWARNING: difference between mean and max_y > 20% = ', abs( abs(mean_7 - max_y) / mean_7 )
        print '\t using peak value'
        mean_7 = max_y
    return mean_7

In [33]:
def peakfinder (x_dat, y_dat, peak_points = 30, tolerance = 1.0E-3, verbose=False):
    
    # Find maximum
    max_y = max(y_dat)
    if verbose: print '\n\tMAX(y_dat) = ', max_y   
    
    # Assume peak as centre
    index = np.where(y_dat == max_y)[0][0]
    if verbose: print '\tMAX(y_dat) index = ', index  
    
    print '\tabs(x_dat[index]) = ', abs(x_dat[index])

    # Manually find 0 index
    index = -1
    final_index = -1
    
    if x_dat[0] > 0.:
        for i in xrange(len(x_dat)):
            index = index + 1     
            if x_dat[i] < 0.0:
                final_index = index
                break
    else:
         for i in xrange(len(x_dat)):
            index = index + 1     
            if x_dat[i] > 0.0:
                final_index = index
                break       

    index = final_index
    if verbose: print '\tindex = ', index       
       
    if verbose: print '\tindex of x_dat = ', x_dat[index],' is ', index
        
    # Find mean around some range of points about the centre
    peak_points_range = (2*peak_points + 1)
    print '\nTaking the average of centre ', peak_points_range, ' points'
    av_indices = np.linspace(-peak_points, peak_points, peak_points_range)
    
    mean_dat = []
    for av_id in av_indices:
        mean_dat.append(y_dat[index+int(av_id)])    
    mean_11 = np.mean(mean_dat)
    if verbose: print '\tMean of ',peak_points_range,' points about centre = ', mean_11
        
    # If the calculated peak is 30% smaller than the absolute peak it is likely the peak is off centre
    # perform the same averaging around the absolute peak
    if abs( abs(mean_11 - max_y) / mean_11 ) > 0.3:
        print '\tWARNING: difference between mean and max_y > 30% = ', abs( abs(mean_11 - max_y) / mean_11 )
        print '\t using peak value'
        #mean_11 = max_y
        index = np.where(y_dat == max_y)[0][0]
        mean_dat = []
        
        for av_id in av_indices:
            mean_dat.append(y_dat[index+int(av_id)])    
        mean_11 = np.mean(mean_dat)
        if verbose: print '\tMean of ',peak_points_range,' points about centre = ', mean_11
        
    return mean_11

## find indices of peak in data

In [21]:
def find_factor_of_peak_indices(x_data, y_data, fac, verbose = False):
    indices = []
    first = int(0)
    second = int(0)    
    
    indx_max = np.argmax(y_data)
    peak = y_data[indx_max]
    
    for i in xrange(len(y_data)):
        if y_data[i] > peak*fac:
            if verbose: print 'half peak of ', y_data[i] , ' found at index ', i
            first = i
            break
        
    for i in reversed(xrange(len(y_data))):
        if y_data[i] > peak*fac:
            if verbose: print 'half peak of ', y_data[i] , ' found at index ', i
            second = i
            break
            
    indices.append(first)
    indices.append(second)
    
    return indices

def find_half_peak_indices(x_data, y_data, verbose = False):
    indices = []
    first = int(0)
    second = int(0)    
    
    indx_max = np.argmax(y_data)
    peak = y_data[indx_max]
    
    for i in xrange(len(y_data)):
        if y_data[i] > peak/2:
            if verbose: print 'half peak of ', y_data[i] , ' found at index ', i
            first = i
            break
        
    for i in reversed(xrange(len(y_data))):
        if y_data[i] > peak/2:
            if verbose: print 'half peak of ', y_data[i] , ' found at index ', i
            second = i
            break
    
    indices.append(first)
    indices.append(second)
    
    return indices

## Plot MD profiles against simulation profiles

In [22]:
def Plot_profiles_MD_cf_Sims(d, df, md_points, vertical = 0, main_label = '', bins=1000, sim_label='Simulation', md_label='MD'):

    # One colour per data point
    colors = cm.rainbow(np.linspace(0, 1, len(df)))

    fig=plt.figure(figsize=(8, 4), dpi= 200, facecolor='w', edgecolor='k');
    multi = 1/1E-3
    sim_norm = 1.0
    
    plt.clf()
    if vertical: x, bins_, p = plt.hist(d['particles']['y'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='k')
    else: x, bins_, p = plt.hist(d['particles']['x'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='k')
           
    for i in md_points:
        
        # Normalisation so all plots peak at y=1
        if vertical: df_norm = 1/peakfinder(np.array(df['position V'].iloc[i]), np.array(df['profile V'].iloc[i]))
        else:        df_norm = 1/peakfinder(np.array(df['position H'].iloc[i]), np.array(df['profile H'].iloc[i]))
        sim_norm = 1/peakfinder(np.array(bins_[:-1]), np.array(x))
        
        lab = md_label + str(i)
        if vertical: plt.plot(df['position V'].iloc[i], np.array(df['profile V'].iloc[i])*df_norm, color=colors[i], label = md_label, lw=0.5)
        else:        plt.plot(df['position H'].iloc[i], np.array(df['profile H'].iloc[i])*df_norm, color=colors[i], label = md_label, lw=0.5)
    
    bins_ = np.array(bins_[:-1]) + (abs(bins_[0]-bins_[1])/2)

    plt.plot(bins_, x*sim_norm, lw=2, color='k', label=sim_label)
    
    
    plt.ylabel('Arbitrary Unit')
    if vertical: plt.xlabel('y [mm]')
    else: plt.xlabel('x [mm]')
    plt.legend(fontsize='x-small')
    plt.grid();
    plt.title(main_label)
    savename = main_label + '.png'
    fig.savefig(savename);
    

## Plot MD profiles againsta  comparison of different space charge method simulations

In [23]:
def Plot_profiles_cf_sc_models(df, md_points, d1, vertical=0, d2=None, d3=None, main_label = '', save_name = 'test', bins=1000, sim_labels=None, md_label='Measurement', plot_md=False):

    # One colour per data point
    colors = cm.rainbow(np.linspace(0, 1, len(df)))

    fig=plt.figure(figsize=(8, 4), dpi= 200, facecolor='w', edgecolor='k');
    multi = 1/1E-3
    
    if sim_labels is None:
        if d2 is not None:
            sim_labels = ['2.5D','nLK']
        elif d3 is not None:
            sim_labels = ['2.5D','nLK','SbS']            
        else:
            sim_labels = ['2.5D']
    
    plt.clf()
    if vertical: x1, bins_1, p1 = plt.hist(d1['particles']['y'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='k')
    else: x1, bins_1, p1 = plt.hist(d1['particles']['x'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='k')
        
    if d2 is not None:
        plt.clf()
        if vertical: x2, bins_2, p2 = plt.hist(d2['particles']['y'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='r')
        else: x2, bins_2, p2 = plt.hist(d2['particles'][
            'x'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='r')
        
        if d3 is not None:
            plt.clf()
            if vertical: x3, bins_3, p3 = plt.hist(d3['particles']['y'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='b')
            else: x3, bins_3, p3 = plt.hist(d3['particles']['x'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='b')
   
        
    norm1 = 1/peakfinder(np.array(bins_1[:-1]), np.array(x1), verbose=True)
    if d2 is not None: norm2 = 1/peakfinder(np.array(bins_2[:-1]), np.array(x2), verbose=True)
    if d3 is not None: norm3 = 1/peakfinder(np.array(bins_3[:-1]), np.array(x3), verbose=True)
        
    if plot_md:
        for i in md_points:
            # Normalisation so all plots peak at y=1
            if vertical:   multi2 = 1/peakfinder(np.array(df0['position V'].iloc[i]), np.array(df0['profile V'].iloc[i]))
            else:          multi2 = 1/peakfinder(np.array(df0['position H'].iloc[i]), np.array(df0['profile H'].iloc[i]))
#             multi2 = 1/max(np.array(df0['profile V'].iloc[i]))

            lab = md_label + str(i)
            if vertical: plt.plot(df0['position V'].iloc[i], np.array(df0['profile V'].iloc[i])*multi2, color='b', label = md_label, lw=1)
            else: plt.plot(df0['position H'].iloc[i], np.array(df0['profile H'].iloc[i])*multi2, color='b', label = md_label, lw=1)

    bins1 = np.array(bins_1[:-1]) + (abs(bins_1[0]-bins_1[1])/2)
    plt.plot(bins1, x1*norm1, lw=2, color='r', label=sim_labels[0])
    if d2 is not None: 
        bins2 = np.array(bins_2[:-1]) + (abs(bins_2[0]-bins_2[1])/2)
        plt.plot(bins2, x2*norm2, lw=2, color='r', label=sim_labels[1])
    if d3 is not None: 
        bins3 = np.array(bins_3[:-1]) + (abs(bins_3[0]-bins_3[1])/2)
        plt.plot(bins3, x3*norm3, lw=2, color='b', label=sim_labels[2])
    
    
    if vertical: plt.xlim(-20, 20)
    else: plt.xlim(-25, 25)
    plt.ylim(-0.05, 1.1)
    
    plt.ylabel('Arbitrary Unit')
    if vertical: plt.xlabel('y [mm]')
    else: plt.xlabel('x [mm]')
    plt.legend(fontsize='x-small')
    plt.grid();
    plt.title(main_label)
    savename = save_name.replace(" ", "_") + '.png'
    fig.savefig(savename);

In [24]:
def Plot_profiles_cf_sc_models_single(df, md_points, d1, vertical=0, main_label = '', save_name = 'test', bins=1000, sim_labels=None, md_label='Measurement', plot_md=False):

    # One colour per data point
    colors = cm.rainbow(np.linspace(0, 1, len(df)))

    fig=plt.figure(figsize=(6, 4), dpi= 200, facecolor='w', edgecolor='k');
    multi = 1/1E-3
    
    if sim_labels is None:
        if d2 is not None:
            sim_labels = ['2.5D','nLK']
        elif d3 is not None:
            sim_labels = ['2.5D','nLK','SbS']            
        else:
            sim_labels = ['2.5D']
    
    plt.clf()
    if vertical: x1, bins_1, p1 = plt.hist(d1['particles']['y'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='k')
    else: x1, bins_1, p1 = plt.hist(d1['particles']['x'][0][0][0]*multi, bins = bins, density=True, histtype=u'step', lw=0, color='k')
        
    norm1 = 1/peakfinder(np.array(bins_1[:-1]), np.array(x1), peak_points = 30, verbose=True)
        
    if plot_md:
        for i in md_points:
            # Normalisation so all plots peak at y=1
            if vertical:   multi2 = 1/peakfinder(np.array(df0['position V'].iloc[i]), np.array(df0['profile V'].iloc[i]), peak_points = 10)
            else:          multi2 = 1/peakfinder(np.array(df0['position H'].iloc[i]), np.array(df0['profile H'].iloc[i]), peak_points = 10)

            lab = md_label + str(i)
            if vertical: plt.plot(df0['position V'].iloc[i], np.array(df0['profile V'].iloc[i])*multi2, color='b', lw=1)
            else: plt.plot(df0['position H'].iloc[i], np.array(df0['profile H'].iloc[i])*multi2, color='b', lw=1)

    bins1 = np.array(bins_1[:-1]) + (abs(bins_1[0]-bins_1[1])/2)
    plt.plot(bins1, x1*norm1, lw=2, color='r')
    
    if vertical: plt.xlim(-20, 20)
    else: plt.xlim(-25, 25)
    plt.ylim(-0.05, 1.1)
    
    plt.ylabel('Arbitrary Unit')
    if vertical: plt.xlabel('y [mm]')
    else: plt.xlabel('x [mm]')
    #plt.legend(fontsize='x-small')
    
    custom_lines = [Line2D([0], [0], color='b', lw=4),Line2D([0], [0], color='r', lw=4)] 
    
    if len(md_points) is 1:
        num_measurements = str(len(md_points)) + ' Measurement'        
    else:
        num_measurements = str(len(md_points)) + ' Measurements'
    plt.legend(custom_lines, [num_measurements, 'LEQ Simulation'], fontsize='small')
    
    plt.grid();
    plt.title(main_label)
    savename = save_name.replace(" ", "_") + '.png'
    fig.savefig(savename);